# Homework_3_applepen

In [325]:
#библиотеки
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt

In [326]:
# файлы для чтения
inv_filename = 'out_input_MS-b1-inventory.csv'
sell_filename = 'out_input_MS-b1-sell.csv'
supp_filename = 'out_input_MS-b1-supply.csv'

In [364]:
# dataframe из файлов
inv_b1 = pd.read_csv(inv_filename, sep = ',',
                     parse_dates = ['date'], dayfirst = True)
sell_b1 = pd.read_csv(sell_filename, sep = ',',
                      parse_dates = ['date'], dayfirst = True)
supp_b1 = pd.read_csv(supp_filename, sep = ',', 
                      parse_dates = ['date'], dayfirst = True)

# 1 задание

In [ ]:
cnt_inv_b1 = 1
cnt_inv_b1 = (sell_b1.date.max() - sell_b1.date.min()).days + 1

date_every_day = pd.date_range(sell_b1.date[0], periods = cnt_inv_b1, freq='D')
np_inv_b1_apple = np.empty(cnt_inv_b1)
np_inv_b1_pen = np.empty(cnt_inv_b1)

i = 0
j = 0
day_apply = 1
day_inventory = 0
checker = False
for i  in range (cnt_inv_b1):    # каждый день
    supply_val_apple = 0
    supply_val_pen = 0
    sum_sell_ap = 0
    sum_sell_pe = 0
    if i != cnt_inv_b1 - 1 :
        if supp_b1.index.max() + 1 != day_apply and sell_b1.date[j] == supp_b1.date[day_apply] :
                    supply_val_apple =  supp_b1.apple[day_apply]
                    supply_val_pen = supp_b1.pen[day_apply]
                    day_apply+=1
                    
        while sell_b1.date[j] == sell_b1.date[j + 1] :
            if sell_b1.sku_num[j][6] == 'a':
                sum_sell_ap+=1
            else : 
                sum_sell_pe+=1
            j+=1
        if sell_b1.sku_num[j][6] == 'a':
                sum_sell_ap+=1
        else : 
                sum_sell_pe+=1
        j+=1
    else :
        while j != sell_b1.index.max() + 1 :
            if sell_b1.sku_num[j][6] == 'a':
                sum_sell_ap+=1
            else : 
                sum_sell_pe+=1
            j+=1
        j-=1
        
    if i != 0 :
        if checker :
            np_inv_b1_apple[i] = inv_b1.apple[day_inventory] - sum_sell_ap + supply_val_apple
            np_inv_b1_pen[i] = inv_b1.pen[day_inventory] - sum_sell_pe + supply_val_pen
            day_inventory+=1
        else :
            np_inv_b1_apple[i] = np_inv_b1_apple[i-1] - sum_sell_ap + supply_val_apple
            np_inv_b1_pen[i] = np_inv_b1_pen[i-1] - sum_sell_pe + supply_val_pen
        
        if j != sell_b1.index.max() and sell_b1.date[j - 1] == inv_b1.date[day_inventory]:
            checker = True
        else :
            checker = False
    else :
        np_inv_b1_apple[i] = supp_b1.apple[0] - sum_sell_ap
        np_inv_b1_pen[i] = supp_b1.pen[0] - sum_sell_pe




In [328]:
# создаем dataframe префайл 1-го задания для 1 штата
df_day = pd.DataFrame({
    'date' : date_every_day,
    'apple' : np_inv_b1_apple.astype(int),
    'pen' : np_inv_b1_pen.astype(int)
})


In [329]:
# Название файла 1 задания
everyday_inventory = 'MS-b1-everyday_inventory.csv'

In [330]:
# Создание файла 1 задания
df_day.to_csv (everyday_inventory, sep = ';', index = False)

# 2 задание

In [350]:
count_steal_b1 = inv_b1.copy(deep = True)
for i in range(inv_b1.index.max() + 1) :
    count_steal_b1.apple[i] = df_day['apple'][df_day.date == inv_b1.date[i]] - inv_b1.apple[i]
    count_steal_b1.pen[i] = df_day['pen'][df_day.date == inv_b1.date[i]] - inv_b1.pen[i]

C:\Users\sanchai\anaconda\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\sanchai\anaconda\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [351]:
# Название файла 2 задания
month_value_stolen = 'MS-b1-month_value_stolen.csv'

In [352]:
# Создание файла 2 задания
count_steal_b1.to_csv(month_value_stolen, sep = ';', index = False)
    

# 3 задание

In [355]:
every_year = pd.date_range(sell_b1.date[0], periods = 10, freq='Y')
count = 10
sum_year_apple = 0
sum_year_pen = 0
apple_stolen = np.zeros(count)
pen_stolen = np.zeros(count)


for i in range(count_steal_b1.index.max() + 1) :
    sum_year_apple = sum_year_apple + count_steal_b1.apple[i]
    sum_year_pen = sum_year_pen + count_steal_b1.pen[i]
    if i % 11 == 0 and i != 0 :
        apple_stolen[i//11 - 1] = sum_year_apple
        pen_stolen[i//11 - 1] = sum_year_pen
        sum_year_apple = 0

In [358]:
count = 10
sum_year_apple = 0
sum_year_pen = 0
apple_sold = np.zeros(count)
pen_sold = np.zeros(count)
e_y = 0

while i != sell_b1.index.max() + 1 :
    if sell_b1.date[i] == every_year[e_y] :
        e_y+=1
        while i != sell_b1.index.max() and sell_b1.date[i] == sell_b1.date[i + 1] :
            if sell_b1.sku_num[i][6] == 'a':
                sum_year_apple+=1
            else :
                sum_year_pen+=1
            i+=1
        if sell_b1.sku_num[i][6] == 'a':
                sum_year_apple+=1
        else :
                sum_year_pen+=1
        i+=1
        apple_sold[e_y - 1] = sum_year_apple 
        pen_sold[e_y - 1] = sum_year_pen
        sum_year_apple = 0
        sum_year_pen = 0
    else :       
        if sell_b1.sku_num[i][6] == 'a':
                sum_year_apple+=1
        else : 
                sum_year_pen+=1
        i+=1

In [361]:
year_agr_1 = pd.DataFrame({
    'year' : every_year,
    'apple_sold' : apple_sold.astype(int),
    'apple_stolen': apple_stolen.astype(int),
    'pen_sold': pen_sold.astype(int),
    'pen_stolen' : pen_stolen.astype(int)
})
year_agr_1

,year,apple_sold,apple_stolen,pen_sold,pen_stolen
0,2006-12-31,681222,78,52555,91
1,2007-12-31,681281,56,51939,159
2,2008-12-31,685110,80,52032,230
3,2009-12-31,682296,69,52411,316
4,2010-12-31,680411,70,52600,392
5,2011-12-31,683083,75,52103,500
6,2012-12-31,685007,74,52578,582
7,2013-12-31,683401,71,52133,660
8,2014-12-31,682371,74,52135,738
9,2015-12-31,681763,79,51804,824


In [362]:
#Название файла 3 задания
year_agr_st_b1 = 'MS-b1-year_agr_state_b1.csv'

In [363]:
#Создание файла 3 задания
year_agr_1.to_csv(year_agr_st_b1, sep = ';', index = False)